In [22]:
import os
import re
import pprint
import pymongo

from xml.etree import ElementTree as ET
from datetime import datetime
from dotenv import dotenv_values
from SPARQLWrapper import SPARQLWrapper, POST, URLENCODED, DIGEST, JSON

In [90]:
env_config = dotenv_values("../environment/.env")

mongo_client = pymongo.MongoClient(
    host=env_config['MONGODB_HOST'],
    port=int(env_config['MONGODB_PORT']),
    username=env_config['MONGO_USER'],
    password=env_config['MONGO_PASSWORD'],
)
mongo_collection = mongo_client["boe_db"]["boe"]

## Investigando las relaciones "anterior" y "posterior"

In [13]:
anteriores = dict()
## get all from collection with a "materias" field of more than 1 element
for item in mongo_collection.find({"anteriores": {"$exists": True, "$not": {"$size": 0}}}):
    for anterior in item["anteriores"]:
        relacion = anterior["relacion"]
        anteriores[relacion["codigo"]] = relacion["texto"]
pprint.pprint(anteriores)

{'201': 'CORRECCIÓN de errores',
 '202': 'CORRECCIÓN de erratas',
 '203': 'CORRIGE errores',
 '210': 'DEROGA',
 '230': 'DEJA SIN EFECTO',
 '231': 'SUSPENDE',
 '247': 'ENMIENDAS',
 '270': 'MODIFICA',
 '300': 'PUBLICA',
 '330': 'CITA',
 '331': 'EN RELACIÓN con',
 '401': 'PRORROGA',
 '404': 'ACTUALIZA',
 '406': 'AMPLÍA',
 '407': 'AÑADE',
 '420': 'APRUEBA',
 '426': 'TRANSPONE',
 '440': 'DE CONFORMIDAD con',
 '470': 'DECLARA',
 '490': 'DESARROLLA',
 '552': 'Recurso promovido contra',
 '693': 'DICTADA'}


In [14]:
posteriores = dict()
## get all from collection with a "materias" field of more than 1 element
for item in mongo_collection.find({"posteriores": {"$exists": True, "$not": {"$size": 0}}}):
    for anterior in item["posteriores"]:
        relacion = anterior["relacion"]
        posteriores[relacion["codigo"]] = relacion["texto"]
pprint.pprint(posteriores)

{'201': 'CORRECCIÓN de errores',
 '202': 'CORRECCIÓN de erratas',
 '203': 'SE CORRIGEN errores',
 '230': 'SE DEJA SIN EFECTO',
 '270': 'SE MODIFICA',
 '300': 'SE PUBLICA',
 '301': 'SE PUBLICA  Acuerdo de convalidación',
 '406': 'SE AMPLÍA',
 '420': 'SE APRUEBA'}


## Programando inserciones

In [3]:
PREFIXES = """
PREFIX  :     <http://www.semanticweb.org/hackathon/ontology/>
PREFIX  owl:  <http://www.w3.org/2002/07/owl#>
PREFIX  rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX  xml:  <http://www.w3.org/XML/1998/namespace>
PREFIX  xsd:  <http://www.w3.org/2001/XMLSchema#>
PREFIX  rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX  dbpedia: <http://dbpedia.org/ontology/>
"""

In [23]:
semantic_tuple = lambda subject, predicate, object: f"{subject} {predicate} {object} ."

In [92]:
# Set up the SPARQL connection to GraphDB
sparql = SPARQLWrapper(
    f"http://{env_config['GRAPHDB_HOST']}:{env_config['GRAPHDB_PORT']}"
    f"/repositories/{env_config['GRAPHDB_REPOSITORY']}/statements"
)
sparql.setMethod(POST)

In [30]:
def to_title_case(s):
    return re.sub(r"\s+", "", s.title())

def to_camel_case(s):
    title_case = to_title_case(s)
    return title_case[0].lower() + title_case[1:]

In [45]:
def insert_tuples(tuples):
    tuple_lines = "\n".join(tuples)
    query = PREFIXES + "INSERT DATA {\n" + tuple_lines + "\n}"
    sparql.setQuery(query=query)
    return sparql.query()

In [62]:
def create_range(name, code):
    entity_name = to_title_case(name)
    tuples = [
        semantic_tuple(f":{entity_name}", "rdf:type", ":owl:NamedIndividual"),
        semantic_tuple(f":{entity_name}", "rdf:type", ":RangoLegislativo"),
        semantic_tuple(f":{entity_name}", ":nombre", '"{name}"^^xsd:string'),
        semantic_tuple(f":{entity_name}", ":codigo", f'"{code}"^^xsd:integer'),
    ]
    return entity_name, tuples

In [55]:
create_range("Otro rango", -2)

[':OtroRango rdf:type :owl:NamedIndividual .',
 ':OtroRango rdf:type :RangoLegislativo .',
 ':OtroRango :nombre "{name}"^^xsd:string .',
 ':OtroRango :codigo "-2"^^xsd:integer .']

**Insertar relación anterior o posterior**
```
:corrigeErrores rdf:type owl:ObjectProperty ;
                rdfs:domain :EntradaBOE ;
                rdfs:range :EntradaBOE .

:corrigeErrores rdf:type owl:NamedIndividual ,
                         :RelacionAnterior ;
                :codigo 203 ;
                :nombre "CORRIGE errores" .
```

In [63]:
def create_relationship_type(name, code, is_previous):
    entity_name = to_camel_case(name)
    tuples = [
        semantic_tuple(f":{entity_name}", "rdf:type", "owl:ObjectProperty"),
        semantic_tuple(f":{entity_name}", "rdfs:domain", ":EntradaBOE"),
        semantic_tuple(f":{entity_name}", "rdfs:range", ":EntradaBOE"),
        semantic_tuple(f":{entity_name}", "rdf:type", "owl:NamedIndividual"),
        semantic_tuple(f":{entity_name}", "rdf:type", ":RelacionAnterior" if is_previous else ":RelacionPosterior"),
        semantic_tuple(f":{entity_name}", ":codigo", f'"{code}"^^xsd:integer'),
        semantic_tuple(f":{entity_name}", ":nombre", f'"{name}"^^xsd:string'),
    ]
    return entity_name, tuples

In [64]:
create_relationship_type("SE PUBLICA  Acuerdo de convalidación", 301, False)

('sePublicaAcuerdoDeConvalidación',
 [':sePublicaAcuerdoDeConvalidación rdf:type owl:ObjectProperty .',
  ':sePublicaAcuerdoDeConvalidación rdfs:domain :EntradaBOE .',
  ':sePublicaAcuerdoDeConvalidación rdfs:range :EntradaBOE .',
  ':sePublicaAcuerdoDeConvalidación rdf:type owl:NamedIndividual .',
  ':sePublicaAcuerdoDeConvalidación rdf:type :RelacionPosterior .',
  ':sePublicaAcuerdoDeConvalidación :codigo "301"^^xsd:integer .',
  ':sePublicaAcuerdoDeConvalidación :nombre "SE PUBLICA  Acuerdo de convalidación"^^xsd:string .'])

Insertar departamentos y materias
```

###  http://www.semanticweb.org/hackathon/ontology#Organización_de_las_Comunidades_Autónomas
:Organización_de_las_Comunidades_Autónomas rdf:type owl:NamedIndividual ;
                                           rdf:type :Materia ;
                                           :codigo 5281 ;
                                           :nombre "Organización de las Comunidades Autónomas" .

###  http://www.semanticweb.org/hackathon/ontology/Ministerio_de_Defensa
:Ministerio_de_Defensa rdf:type owl:NamedIndividual ,
                                :Departamento ;
                       :codigo 6110 ;
                       :nombre "Ministerio de Defensa" .
```

In [65]:
def create_topic(name, code):
    entity_name = to_title_case(name)
    tuples = [
        semantic_tuple(f":{entity_name}", "rdf:type", ":owl:NamedIndividual"),
        semantic_tuple(f":{entity_name}", "rdf:type", ":Materia"),
        semantic_tuple(f":{entity_name}", ":nombre", f'"{name}"^^xsd:string'),
        semantic_tuple(f":{entity_name}", ":codigo", f'"{code}"^^xsd:integer'),
    ]
    return entity_name, tuples

def create_department(name, code):
    entity_name = to_title_case(name)
    tuples = [
        semantic_tuple(f":{entity_name}", "rdf:type", ":owl:NamedIndividual"),
        semantic_tuple(f":{entity_name}", "rdf:type", ":Departamento"),
        semantic_tuple(f":{entity_name}", ":nombre", f'"{name}"^^xsd:string'),
        semantic_tuple(f":{entity_name}", ":codigo", f'"{code}"^^xsd:integer'),
    ]
    return entity_name, tuples

Entradas BOE
```
###  http://www.semanticweb.org/hackathon/ontology/BOE-A-2022-22147
:BOE-A-2022-22147 rdf:type owl:NamedIndividual .


###  http://www.semanticweb.org/hackathon/ontology/BOE-A-2023-17679
:BOE-A-2023-17679 rdf:type owl:NamedIndividual ,
                           :EntradaBOE ;
                  :corrigeErrores :BOE-A-2022-22147 ;
                  :departamento :Ministerio_de_Defensa ;
                  :origenLegislativo <http://dbpedia.org/resource/España> ;
                  :rangoLegislativo :Resolución ;
                  :fechaPublicacion "2023-08-01T00:00:00.000Z"^^xsd:dateTime ;
                  :identificador "BOE-A-2023-17679" ;
                  :titulo "Resolución 400/38323/2023, de 21 de julio, de la Subsecretaría, por la que se corrigen errores y se modifica la Resolución 400/38495/2022, de 21 de diciembre, por la que se convoca proceso selectivo para la estabilización de empleo temporal, para el acceso a la condición de personal estatutario fijo en plazas de la categoría de Enfermero/a, Fisioterapeuta y Matrona en la Red Hospitalaria de la Defensa." .

```

In [66]:
def create_named_individual(name):
    tuples = [
        semantic_tuple(f":{name}", "rdf:type", ":owl:NamedIndividual"),
    ]
    return name, tuples

In [72]:
def create_boe_entry(title, code, date, department, topics, previous, posteriors):
    entity_name = code
    tuples = [
        semantic_tuple(f":{entity_name}", "rdf:type", "owl:NamedIndividual"),
        semantic_tuple(f":{entity_name}", "rdf:type", ":EntradaBOE"),
        semantic_tuple(f":{entity_name}", ":titulo", f'"{title}"^^xsd:string'),
        semantic_tuple(f":{entity_name}", ":identificador", f'"{code}"^^xsd:string'),
        semantic_tuple(f":{entity_name}", ":fechaPublicacion", f'"{date}"^^xsd:date'),
    ]

    if department is not None: 
        department_entity_name, department_tuples = create_department(department["texto"], department["codigo"])
        tuples.extend(department_tuples)
        tuples.append(semantic_tuple(f":{entity_name}", ":departamento", f":{department_entity_name}"))
    
    # TODO: origen legislativo, debemos buscarlo en otra DB
    
    for topic in topics:
        topic_entity_name, topic_tuples = create_topic(topic["texto"], topic["codigo"])
        tuples.extend(topic_tuples)
        tuples.append(semantic_tuple(f":{entity_name}", ":materia", f":{topic_entity_name}"))
    for prev in previous:
        prev_entity_name, prev_tuples = create_relationship_type(prev["relacion"]["texto"], prev["relacion"]["codigo"], True)
        related_code = prev["identificador"]
        tuples.extend(prev_tuples)
        tuples.append(semantic_tuple(f":{entity_name}", f":{prev_entity_name}", f":{related_code}"))
    for post in posteriors:
        post_entity_name, post_tuples = create_relationship_type(post["relacion"]["texto"], post["relacion"]["codigo"], False)
        related_code = post["identificador"]
        tuples.extend(post_tuples)
        tuples.append(semantic_tuple(f":{entity_name}", f":{post_entity_name}", f":{related_code}"))

    return entity_name, tuples

In [75]:
pprint.pprint(create_boe_entry(
    "Prueba", "BOE-A-2021-1234", "2021-01-01",
    {"codigo": 1, "texto": "Ministerio de la Verdad"},
    [{"codigo": 1, "texto": "Materia 1"}, {"codigo": 2, "texto": "Materia 2"}],
    [{"identificador": "BOE-A-2021-1233", "relacion": {"codigo": 1, "texto": "Relacion 1"}}],
    [{"identificador": "BOE-A-2021-1235", "relacion": {"codigo": 2, "texto": "Relacion 2"}}],
))

('BOE-A-2021-1234',
 [':BOE-A-2021-1234 rdf:type owl:NamedIndividual .',
  ':BOE-A-2021-1234 rdf:type :EntradaBOE .',
  ':BOE-A-2021-1234 :titulo "Prueba"^^xsd:string .',
  ':BOE-A-2021-1234 :identificador "BOE-A-2021-1234"^^xsd:string .',
  ':BOE-A-2021-1234 :fechaPublicacion "2021-01-01"^^xsd:date .',
  ':MinisterioDeLaVerdad rdf:type :owl:NamedIndividual .',
  ':MinisterioDeLaVerdad rdf:type :Departamento .',
  ':MinisterioDeLaVerdad :nombre "Ministerio de la Verdad"^^xsd:string .',
  ':MinisterioDeLaVerdad :codigo "1"^^xsd:integer .',
  ':BOE-A-2021-1234 :departamento :MinisterioDeLaVerdad .',
  ':Materia1 rdf:type :owl:NamedIndividual .',
  ':Materia1 rdf:type :Materia .',
  ':Materia1 :nombre "Materia 1"^^xsd:string .',
  ':Materia1 :codigo "1"^^xsd:integer .',
  ':BOE-A-2021-1234 :materia :Materia1 .',
  ':Materia2 rdf:type :owl:NamedIndividual .',
  ':Materia2 rdf:type :Materia .',
  ':Materia2 :nombre "Materia 2"^^xsd:string .',
  ':Materia2 :codigo "2"^^xsd:integer .',
  ':BO

In [78]:
entity_name, tuples = create_boe_entry(
    "Prueba", "BOE-A-2021-1234", "2021-01-01",
    {"codigo": 1, "texto": "Ministerio de la Verdad"},
    [{"codigo": 1, "texto": "Materia 1"}, {"codigo": 2, "texto": "Materia 2"}],
    [{"identificador": "BOE-A-2021-1233", "relacion": {"codigo": 1, "texto": "Relacion 1"}}],
    [{"identificador": "BOE-A-2021-1235", "relacion": {"codigo": 2, "texto": "Relacion 2"}}],
)
insert_tuples(tuples).info()

{'vary': 'Accept-Encoding',
 'cache-control': 'no-store',
 'content-language': 'en-US',
 'date': 'Fri, 10 Nov 2023 18:42:45 GMT',
 'connection': 'close',
 'server': 'GraphDB/10.4.1 RDF4J/4.3.6'}

## Insertando desde MongoDB

In [87]:
for document in mongo_collection.find({}):
    title = document["titulo"]
    code = document["identificador"]
    date_str = document["fecha_publicacion"].strftime("%Y-%m-%d")
    department = document.get("departamento")
    topics = document.get("materias")
    previous = document.get("anteriores")
    posteriors = document.get("posteriores")
    entity_name, tuples = create_boe_entry(title, code, date_str, department, topics, previous, posteriors)
    pprint.pprint(tuples)
    # print(insert_tuples(tuples).info())
    break

[':BOE-B-2023-32033 rdf:type owl:NamedIndividual .',
 ':BOE-B-2023-32033 rdf:type :EntradaBOE .',
 ':BOE-B-2023-32033 :titulo "Anuncio de la Confederación Hidrográfica del '
 'Duero, O.A., de información pública de una modificación de características '
 'de una concesión de un aprovechamiento de aguas subterráneas para riego, en '
 'el término municipal de Valbuena de Duero (Valladolid)."^^xsd:string .',
 ':BOE-B-2023-32033 :identificador "BOE-B-2023-32033"^^xsd:string .',
 ':BOE-B-2023-32033 :fechaPublicacion "2023-11-03"^^xsd:date .',
 ':MinisterioParaLaTransiciónEcológicaYElRetoDemográfico rdf:type '
 ':owl:NamedIndividual .',
 ':MinisterioParaLaTransiciónEcológicaYElRetoDemográfico rdf:type '
 ':Departamento .',
 ':MinisterioParaLaTransiciónEcológicaYElRetoDemográfico :nombre "Ministerio '
 'para la Transición Ecológica y el Reto Demográfico"^^xsd:string .',
 ':MinisterioParaLaTransiciónEcológicaYElRetoDemográfico :codigo '
 '"9575"^^xsd:integer .',
 ':BOE-B-2023-32033 :departament